In [9]:
import tabula
import pandas as pd
import re

pdf_path = 'MDS2024_25.pdf'

# Extract all tables from the PDF with the lattice option to handle table borders better
try:
    tables_json = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True, output_format="json", encoding='ISO-8859-1', lattice=True)
except Exception as e:
    print(f"Error reading PDF: {e}")
    tables_json = []  # Initialize as empty if reading fails

# Regular expression pattern to identify "Course Outcomes" table
pattern = re.compile(r"Course Outcomes", re.IGNORECASE)

# Filter JSON data for tables with "Course Outcomes"
course_outcomes_tables = []
for table_json in tables_json:
    try:
        # Convert JSON to DataFrame
        table_data = [[cell['text'] for cell in row] for row in table_json['data']]
        table = pd.DataFrame(table_data)
        
        # Check for the pattern in the DataFrame content
        if table.apply(lambda row: row.astype(str).str.contains(pattern).any(), axis=1).any():
            # Set the first row as header if it matches the expected format
            if 'No.' in table.iloc[0].values[0] and 'Course Outcomes' in table.iloc[0].values[1]:
                table.columns = table.iloc[0]  # Set first row as header
                table = table.drop(0).reset_index(drop=True)  # Drop the header row from data
            
            # Standardize column names
            table.columns = ['No', 'Course Outcomes', 'LRNG Needs', 'Unused_1', 'Unused_2'][:table.shape[1]]
            
            # Drop unnecessary columns by name
            table = table.drop(columns=['LRNG Needs', 'Unused_1', 'Unused_2'], errors='ignore')
            
            # Fix merged words in 'Course Outcomes' by adding spaces before capital letters or numbers
            def fix_merged_words(text):
                # Add spaces between lowercase-uppercase or letter-number transitions
                return re.sub(r'(?<=[a-z])(?=[A-Z0-9])', ' ', text)
            
            table['Course Outcomes'] = table['Course Outcomes'].apply(lambda x: fix_merged_words(x))

            # Add the table to the list
            course_outcomes_tables.append(table)
    except Exception as e:
        print(f"Error processing table JSON data: {e}")

# Concatenate all "Course Outcomes" tables into a single DataFrame
if course_outcomes_tables:
    combined_df = pd.concat(course_outcomes_tables, ignore_index=True)
    
    # Drop duplicate rows
    combined_df = combined_df.drop_duplicates()
    
    # Add "Verbs" and "Assessments" columns with empty values
    combined_df['Verbs'] = ""       # Or provide a default value if needed
    combined_df['Assessments'] = ""  # Or provide a default value if needed
    
    # Save to Excel file
    combined_df.to_excel("course_outcomes_combined_single_sheet.xlsx", index=False)
    print("Filtered course outcomes tables have been saved to 'course_outcomes_combined_single_sheet.xlsx'")
else:
    print("No tables containing 'Course Outcomes' found.")


Filtered course outcomes tables have been saved to 'course_outcomes_combined_single_sheet.xlsx'
